In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
m_path = os.path.abspath(os.path.join("..\.."))
if m_path not in sys.path:
    sys.path.append(m_path)
    
from rigging.weight_transfer import main
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [37]:
from smplpytorch.pytorch.smpl_layer import SMPL_Layer

smpl_layer = SMPL_Layer(
    center_idx=0,
    gender='male',
    model_root="C:/Users/User/CODE/FYP/models/smpl",
)

tgt_weights = smpl_layer.smpl_data["weights"]

In [38]:
import pickle
import torch

src_dict_path = "../no_git_test_data/1-1/1-1.pkl"

with open(src_dict_path, "rb") as f:
    src_dict = pickle.load(f)

pose = src_dict['pose']
trans = src_dict['trans']
scale = src_dict['scale']

fin_pose= torch.FloatTensor(pose).unsqueeze(0)
# fin_pose = fin_pose.cuda()

ret_verts, ret_joints = smpl_layer(fin_pose)

ret_verts = ret_verts.detach().cpu().numpy()[0]
ret_joints = ret_joints.detach().cpu().numpy()[0]

trans_verts = ret_verts * scale + trans
trans_joints = ret_joints * scale + trans

smpl_mesh = o3d.geometry.TriangleMesh()
smpl_mesh.vertices = o3d.utility.Vector3dVector(trans_verts)
smpl_mesh.triangles = o3d.utility.Vector3iVector(smpl_layer.th_faces.detach().cpu().numpy())
smpl_mesh.compute_vertex_normals()

joints = trans_joints
joints_pt = o3d.geometry.PointCloud()
joints_pt.points = o3d.utility.Vector3dVector(joints)
joints_pt.colors = o3d.utility.Vector3dVector(np.repeat([[0, 0, 1]], repeats=joints.shape[0], axis=0))

In [7]:
source_mesh_path = "../no_git_test_data/1-1/smpl1.obj"
target_mesh_path = "../no_git_test_data/1-1/model_cleaned.obj"

src_mesh = o3d.io.read_triangle_mesh(source_mesh_path)
tgt_mesh = o3d.io.read_triangle_mesh(target_mesh_path)

src_mesh.compute_vertex_normals()

weights, v_match, v_no_match = main(src_mesh, tgt_mesh, tgt_weights)

In [8]:
colors = np.zeros((weights.shape[0], 3))
colors[v_match] = np.array([0., .5, 0.])
colors[v_no_match] = np.array([.5, 0., 0])
tgt_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

In [29]:
o3d.visualization.draw_geometries([joints_pt])

In [39]:
from xcloth.components.utils import create_o3d_pcd

pcd = create_o3d_pcd(tgt_weights @ joints)
o3d.visualization.draw_geometries([pcd])